# Running our Ping Pong test

I am going to run the ping pong test with the mvapich2 MPI implementation:

In [ ]:
module unload cse6230
module load cse6230/gcc-omp-gpu

In [ ]:
which mpicc

How many cores do I have?

In [ ]:
echo ${PBS_NP}

Where are they?

In [ ]:
cat ${PBS_NODEFILE} | uniq

In [ ]:
make clean
make osu_latency

Let's run the basic test:

In [ ]:
mpirun -np 2 ./osu_latency

**What are our estimates for the latency and bandwidth from this? What is the inverse bandwidth?**

Those numbers sound similar to memory bandwidth numbers, so it sounds like MPI is taking advantage of direct memcpy between processes.  Is that true?

In [ ]:
mpirun -np 2 -env MV2_USE_SHARED_MEM 0 ./osu_latency

Let's make sure we have control over where processes go with pinning (like OpenMP affinity):

In [ ]:
mpirun -np 2 -env MV2_USE_SHARED_MEM 1 -env MV2_ENABLE_AFFINITY 1 ./osu_latency

Now what if I run with 28 processes (the number of cores on one node)? 112 (the number of cores on all nodes)?

In [ ]:
mpirun -np 28 -env MV2_USE_SHARED_MEM 1 -env MV2_ENABLE_AFFINITY 1 ./osu_latency | tail -n 25

In [ ]:
mpirun -np 112 -env MV2_USE_SHARED_MEM 1 -env MV2_ENABLE_AFFINITY 1 ./osu_latency | tail -n 25

All of the ping-pongs so far have been between rank 0 and rank 1.  With affinity, these are in the same NUMA domain.  What if I ping-pong between other ranks?

In [ ]:
mpirun -np 112 -env MV2_USE_SHARED_MEM 1 -env MV2_ENABLE_AFFINITY 1 ./osu_latency 0 14 | tail -n 25

In [ ]:
mpirun -np 112 -env MV2_USE_SHARED_MEM 1 -env MV2_ENABLE_AFFINITY 1 ./osu_latency 0 28 | tail -n 25

In [ ]:
mpirun -np 112 -env MV2_USE_SHARED_MEM 1 -env MV2_ENABLE_AFFINITY 1 ./osu_latency 0 111 | tail -n 25

In [ ]:
mpirun -np 112 -env MV2_USE_SHARED_MEM 1 -env MV2_ENABLE_AFFINITY 1 ./osu_latency 27 111 | tail -n 25